In [1]:
import astropy
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import subprocess
import os

In [2]:
def pop_list(ls,indices):
    for n,ind in enumerate(indices):
        ls.pop(ind)
    return(ls)

In [3]:
photz = Table.read('data/photz/LH_opt_spitzer_merged_vac_opt3as_irac4as_all_hpx_public.fits')
#photz =Table.read('/beegfs/lofar/deepfields/science_ready_catalogs/LH_opt_spitzer_merged_vac_opt3as_irac4as_all_hpx_public.fits')

In [4]:
photz[0]

id,ID_OPTICAL,ID_SPITZER,RA,DEC,help_id,RA_HELP,DEC_HELP,EBV,Z_SPEC,CLASS_STAR,FLAG_CLEAN,u_flux,u_fluxerr,g_flux,g_fluxerr,r_flux,r_fluxerr,z_flux,z_fluxerr,g_rcs_flux,g_rcs_fluxerr,r_rcs_flux,r_rcs_fluxerr,i_rcs_flux,i_rcs_fluxerr,z_rcs_flux,z_rcs_fluxerr,J_flux,J_fluxerr,K_flux,K_fluxerr,ch1_servs_flux,ch1_servs_fluxerr,ch2_servs_flux,ch2_servs_fluxerr,ch1_swire_flux,ch1_swire_fluxerr,ch2_swire_flux,ch2_swire_fluxerr,ch3_swire_flux,ch3_swire_fluxerr,ch4_swire_flux,ch4_swire_fluxerr,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24,F_PACS_100,FErr_PACS_100_u,FErr_PACS_100_l,F_PACS_160,FErr_PACS_160_u,FErr_PACS_160_l,Bkg_PACS_100,Bkg_PACS_160,Sig_conf_PACS_100,Sig_conf_PACS_160,Rhat_PACS_100,Rhat_PACS_160,n_eff_PACS_100,n_eff_PACS_160,Pval_res_100,Pval_res_160,flag_PACS_100,flag_PACS_160,F_SPIRE_250,FErr_SPIRE_250_u,FErr_SPIRE_250_l,F_SPIRE_350,FErr_SPIRE_350_u,FErr_SPIRE_350_l,F_SPIRE_500,FErr_SPIRE_500_u,FErr_SPIRE_500_l,Bkg_SPIRE_250,Bkg_SPIRE_350,Bkg_SPIRE_500,Sig_conf_SPIRE_250,Sig_conf_SPIRE_350,Sig_conf_SPIRE_500,Rhat_SPIRE_250,Rhat_SPIRE_350,Rhat_SPIRE_500,n_eff_SPIRE_250,n_eff_SPIRE_500,n_eff_SPIRE_350,Pval_res_250,Pval_res_350,Pval_res_500,flag_spire_250,flag_spire_350,flag_spire_500,AGN,optAGN,IRAGN,XrayAGN,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,nfilt_eazy,nfilt_atlas,nfilt_ananna,chi_r_best,chi_r_stellar,stellar_type,hp_idx_11,hp_depth_cluster
,,,deg,deg,,deg,deg,,,,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,MJy / sr,MJy / sr,,,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,
int64,int32,int32,float64,float64,bytes27,float64,float64,float32,float64,float32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,bool,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,bytes6,int64,int32
0,1,--,164.48053950361853,55.8801032089467,N/A,nan,nan,0.008567253,-99.0,0.8284701,1,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,1.1428666,0.16968247,1.6765456,0.21724406,1.3065643,0.4575452,2.3796828,1.7248636,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,nan,nan,nan,nan,nan,nan,nan,nan,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,True,0,0.0,0.0,0.0,1.8755,0.0,4.81,0.806,-99.0,-99.0,-99.0,-99.0,-99,-99,-99,-99.0,-99.0,,6228652,33


In [18]:
cigale_input = photz.copy()

In [23]:
filters_file = open('data/photz/Lockman-SWIRE_filters.res.info','r')
#filters_file = open('/beegfs/lofar/deepfields/science_ready_catalogs/Lockman-SWIRE_filters.res.info','r')
filters = []
for line in filters_file:
    temp = line.split(' ')
    filters.append(temp)
filters_file.close()

for n,line in enumerate(filters):
    if n!=32:
        filters[n] = line[0:-1]
    else:
        filters[n][-1] = filters[n][-1].replace('\n','')
    filters[n] = pop_list(filters[n],[1 for m in range(len(filters[n])-2)])
    filters[n][-1] = filters[n][-1].replace('.filter','')
#ignore the first two filters as they are SWIRE channel 1 and 2 and i want to use SERVS channel 1 and 2 as
#they have deeper observations
filters = filters[2:]

In [25]:
filters[2:]

[['3', 'IRAC.I3.res'],
 ['4', 'IRAC.I4.res'],
 ['5', 'MegaCam.g.rcs.res'],
 ['6', 'MegaCam.g.res'],
 ['7', 'MegaCam.i.res'],
 ['8', 'MegaCam.i_0.res'],
 ['9', 'MegaCam.r.rcs.res'],
 ['10', 'MegaCam.r.res'],
 ['11', 'MegaCam.u.res'],
 ['12', 'MegaCam.z.rcs.res'],
 ['13', 'MegaCam.z.res'],
 ['14', 'PS1.g.res'],
 ['15', 'PS1.i.res'],
 ['16', 'PS1.r.res'],
 ['17', 'PS1.y.res'],
 ['18', 'PS1.z.res'],
 ['19', 'UKIDSS.J.res'],
 ['20', 'UKIDSS.K.res'],
 ['21', 'WFC.Gunn_g_qe.res'],
 ['22', 'WFC.Gunn_i_qe.res'],
 ['23', 'WFC.Gunn_r_qe.res'],
 ['24', 'WFC.RGO_u_qe.res'],
 ['25', 'WFC.RGO_z_qe.res'],
 ['26', 'IRAC.I1.res'],
 ['27', 'IRAC.I2.res']]

In [14]:
#translate_file = open('/beegfs/lofar/deepfields/science_ready_catalogs/LH.filter.translate','r')
translate_file = open('data/photz/LH.filter.translate','r')
translate = []
for line in translate_file:
    if '#' in line:
        continue
    temp = line.split(' ')
    translate.append(temp)
translate_file.close()

translate = translate[:len(translate)-1]

for n,line in enumerate(translate):
    translate[n][1] = line[1][1:-1]

In [20]:
translate

[['u_flux', '11'],
 ['u_fluxerr', '11'],
 ['g_flux', '6'],
 ['g_fluxerr', '6'],
 ['r_flux', '10'],
 ['r_fluxerr', '10'],
 ['z_flux', '13'],
 ['z_fluxerr', '13'],
 ['g_rcs_flux', '5'],
 ['g_rcs_fluxerr', '5'],
 ['r_rcs_flux', '9'],
 ['r_rcs_fluxerr', '9'],
 ['i_rcs_flux', '7'],
 ['i_rcs_fluxerr', '7'],
 ['z_rcs_flux', '12'],
 ['z_rcs_fluxerr', '12'],
 ['J_flux', '19'],
 ['J_fluxerr', '19'],
 ['K_flux', '20'],
 ['K_fluxerr', '20'],
 ['ch1_servs_flux', '26'],
 ['ch1_servs_fluxerr', '26'],
 ['ch2_servs_flux', '27'],
 ['ch2_servs_fluxerr', '27'],
 ['ch1_swire_flux', '1'],
 ['ch1_swire_fluxerr', '1'],
 ['ch2_swire_flux', '2'],
 ['ch2_swire_fluxerr', '2'],
 ['ch3_swire_flux', '3'],
 ['ch3_swire_fluxerr', '3'],
 ['ch4_swire_flux', '4'],
 ['ch4_swire_fluxerr', '4']]

In [19]:
for n,line in enumerate(translate):
    colname = line[0]
    filt_num = int(line[1])
    filt_name = ''
    for m,filt in enumerate(filters):
        if m+1==filt_num:
            print(filt)
            filt_name = filt[1]
    
    if 'err' in colname:
        filt_name = filt_name+'_err'
    #print(colname)
    #print(filt_name)
    print(colname)
    print(filt_name)
    cigale_input.rename_column(colname,filt_name)

['11', 'MegaCam.u.res']
u_flux
MegaCam.u.res
['11', 'MegaCam.u.res']
u_fluxerr
MegaCam.u.res_err
['6', 'MegaCam.g.res']
g_flux
MegaCam.g.res
['6', 'MegaCam.g.res']
g_fluxerr
MegaCam.g.res_err
['10', 'MegaCam.r.res']
r_flux
MegaCam.r.res
['10', 'MegaCam.r.res']
r_fluxerr
MegaCam.r.res_err
['13', 'MegaCam.z.res']
z_flux
MegaCam.z.res
['13', 'MegaCam.z.res']
z_fluxerr
MegaCam.z.res_err
['5', 'MegaCam.g.rcs.res']
g_rcs_flux
MegaCam.g.rcs.res
['5', 'MegaCam.g.rcs.res']
g_rcs_fluxerr
MegaCam.g.rcs.res_err
['9', 'MegaCam.r.rcs.res']
r_rcs_flux
MegaCam.r.rcs.res
['9', 'MegaCam.r.rcs.res']
r_rcs_fluxerr
MegaCam.r.rcs.res_err
['7', 'MegaCam.i.res']
i_rcs_flux
MegaCam.i.res
['7', 'MegaCam.i.res']
i_rcs_fluxerr
MegaCam.i.res_err
['12', 'MegaCam.z.rcs.res']
z_rcs_flux
MegaCam.z.rcs.res
['12', 'MegaCam.z.rcs.res']
z_rcs_fluxerr
MegaCam.z.rcs.res_err
['19', 'UKIDSS.J.res']
J_flux
UKIDSS.J.res
['19', 'UKIDSS.J.res']
J_fluxerr
UKIDSS.J.res_err
['20', 'UKIDSS.K.res']
K_flux
UKIDSS.K.res
['20', 'UKIDSS.K

KeyError: 'Column  already exists'

In [140]:
cigale_input.rename_column('z1_median','redshift')

ID,ID_OPTICAL,ID_SPITZER,RA,DEC,help_id,RA_HELP,DEC_HELP,EBV,Z_SPEC,Z_SOURCE,Z_QUAL,RA_ZSPEC,DEC_ZSPEC,REL,OBJID,CLASS_STAR,FLAG_DEEP,FLAG_OVERLAP,FLAG_CLEAN,FUV_flux,FUV_fluxerr,NUV_flux,NUV_fluxerr,LBCBlue_Uspec,LBCBlue_Uspec_err,Bw_atmccd,Bw_atmccd_err,R_atmccd,R_atmccd_err,I_atmccd,I_atmccd_err,sdssz2,sdssz2_err,suprime_FDCCD_z,suprime_FDCCD_z_err,LBCR_Y,LBCR_Y_err,jband,jband_err,hband,hband_err,K_atmccd,K_atmccd_err,kband,kband_err,IRAC_ch1_total_response,IRAC_ch1_total_response_err,IRAC_ch2_total_response,IRAC_ch2_total_response_err,IRAC_ch3_total_response,IRAC_ch3_total_response_err,IRAC_ch4_total_response,IRAC_ch4_total_response_err,F_MIPS_24,FErr_MIPS_24_u,FErr_MIPS_24_l,Bkg_MIPS_24,Sig_conf_MIPS_24,Rhat_MIPS_24,n_eff_MIPS_24,Pval_res_24,flag_mips_24,F_PACS_100,FErr_PACS_100_u,FErr_PACS_100_l,F_PACS_160,FErr_PACS_160_u,FErr_PACS_160_l,Bkg_PACS_100,Bkg_PACS_160,Sig_conf_PACS_100,Sig_conf_PACS_160,Rhat_PACS_100,Rhat_PACS_160,n_eff_PACS_100,n_eff_PACS_160,Pval_res_100,Pval_res_160,flag_PACS_100,flag_PACS_160,F_SPIRE_250,FErr_SPIRE_250_u,FErr_SPIRE_250_l,F_SPIRE_350,FErr_SPIRE_350_u,FErr_SPIRE_350_l,F_SPIRE_500,FErr_SPIRE_500_u,FErr_SPIRE_500_l,Bkg_SPIRE_250,Bkg_SPIRE_350,Bkg_SPIRE_500,Sig_conf_SPIRE_250,Sig_conf_SPIRE_350,Sig_conf_SPIRE_500,Rhat_SPIRE_250,Rhat_SPIRE_350,Rhat_SPIRE_500,n_eff_SPIRE_250,n_eff_SPIRE_500,n_eff_SPIRE_350,Pval_res_250,Pval_res_350,Pval_res_500,flag_spire_250,flag_spire_350,flag_spire_500,AGN,optAGN,IRAGN,AGN_ZSPEC,XrayAGN,XrayFlux_0.5-2,XrayHardness,z1_median,z1_min,z1_max,z1_area,z2_median,z2_min,z2_max,z2_area,nfilt_eazy,nfilt_atlas,nfilt_ananna,chi_r_best,chi_r_stellar,stellar_type,hp_idx_11,hp_depth_cluster
,,,deg,deg,,deg,deg,,,,,,,,,,,,,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,uJy,muJy,muJy,muJy,MJy / sr,MJy / sr,,,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,mJy/Beam,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,deg,
int64,int64,int64,float64,float64,bytes27,float64,float64,float32,float64,int64,int64,float64,float64,float64,bytes25,float32,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,bool,bool,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,bytes6,int64,int32
0,1,--,216.473863,32.2825963,N/A,nan,nan,0.01093379,nan,--,--,nan,nan,nan,N/A,0.64,0,0,1,-99.0,-99.0,-99.0,-99.0,0.10352171485738193,0.09050546420954841,-99.0,-99.0,-99.0,-99.0,1.2489124884509977,0.17990861652974743,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-0.49297147310738154,0.9599805695282839,0.20943547943156204,1.134281811727078,7.514107605812171,4.5540334671514415,7.615719616676927,4.989334194948697,nan,nan,nan,nan,nan,nan,nan,nan,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,True,0,0.0,0.0,--,0.0,-99.0,-99.0,3.6887,2.88,4.63,0.379,0.3205,0.0,0.81,0.3,6,6,-99,0.9076026,-99.0,,9355052,38
1,2,--,216.6176098,32.2827433,N/A,nan,nan,0.011378967,nan,--,--,nan,nan,nan,N/A,0.66,1,0,1,-99.0,-99.0,-99.0,-99.0,0.14205340354133433,0.102083831

In [ ]:
Table.write(cigale_input,'cigale_input_Lockman.fits',format='fits',overwrite=True)